## Initialization

In [ ]:
!git clone https://git.ecker.tech/mrq/ai-voice-cloning/
%cd ai-voice-cloning

!git submodule init
!git submodule update --remote

# TODO: fix venvs working for subprocess.Popen calling a bash script
#!apt install python3.8-venv
#!python -m venv venv
#!source ./venv/bin/activate

!python -m pip install --upgrade pip
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116
!python -m pip install -r ./dlas/requirements.txt
!python -m pip install -r ./tortoise-tts/requirements.txt
!python -m pip install -r ./requirements.txt
!python -m pip install -e ./tortoise-tts/

!rm ./tortoise-tts/{main,webui}.py

# Update Repos

In [ ]:
# for my debugging purposes
%cd /content/ai-voice-cloning/
!./update.sh

# Mount Drive

In [ ]:
# only run once, this will save all userdata to your Drive
# it shouldn't delete through symlinks, but you never know

from google.colab import drive

%cd /content/ai-voice-cloning
drive.flush_and_unmount()
!rm -r ./{training,results,voices,config}
drive.mount('/content/drive')
!mkdir /content/drive/MyDrive/ai-voice-cloning/
!mv /content/drive/MyDrive/{training,results,voices,config} /content/drive/MyDrive/ai-voice-cloning
!mkdir /content/drive/MyDrive/ai-voice-cloning/{training,results,voices,config}
!ln -s /content/drive/MyDrive/ai-voice-cloning/{training,results,voices,config} ./

## Running (Inlined)

In [ ]:
%cd /content/ai-voice-cloning
#!source ./venv/bin/activate

import os
import sys

sys.argv = [""]
if './src/' not in sys.path:
	sys.path.append('./src/')
if './tortoise-tts/' not in sys.path:
	sys.path.append('./tortoise-tts/')

if 'TORTOISE_MODELS_DIR' not in os.environ:
	os.environ['TORTOISE_MODELS_DIR'] = os.path.realpath(os.path.join(os.getcwd(), './models/tortoise/'))

if 'TRANSFORMERS_CACHE' not in os.environ:
	os.environ['TRANSFORMERS_CACHE'] = os.path.realpath(os.path.join(os.getcwd(), './models/transformers/'))

import utils
import webui

args = utils.setup_args()
ui = webui.setup_gradio()
# Be very, very sure to check "Defer TTS Load" in Settings, then restart, before you start training
# You'll crash the runtime if you don't
if not args.defer_tts_load:
	utils.setup_tortoise()

ui.launch(share=True, prevent_thread_lock=True, height=1000)
ui.block_thread()

## Running (non-inlined)

In [ ]:
%cd /content/ai-voice-cloning/
!./start.sh

# Restart Runtime

In [ ]:
!rm ./tortoise-tts/{main,webui}.py
exit()

# Fallback Training

In [ ]:
# This is in case you can't get training through the web UI
%cd /content/ai-voice-cloning
!python ./dlas/codes/train.py -opt ./training/finetune.yaml

## Exporting

In [ ]:
# if you're not using drive mounting
%cd /content/ai-voice-cloning
!apt install -y p7zip-full
from datetime import datetime
timestamp = datetime.now().strftime('%m-%d-%Y_%H:%M:%S')
!mkdir -p "../{timestamp}/results"
!mv ./results/* "../{timestamp}/results/."
!mv ./training/* "../{timestamp}/training/."
!7z a -t7z -m0=lzma2 -mx=9 -mfb=64 -md=32m -ms=on "../{timestamp}.7z" "../{timestamp}/"
!ls ~/
!echo "Finished zipping, archive is available at {timestamp}.7z"